# Language Translation
In this project, you’re going to take a peek into the realm of neural network machine translation.  You’ll be training a sequence to sequence model on a dataset of English and French sentences that can translate new sentences from English to French.
## Get the Data
Since translating the whole language of English to French will take lots of time to train, we have provided you with a small portion of the English corpus.

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import problem_unittests as tests

source_path = 'data/small_vocab_en'
target_path = 'data/small_vocab_fr'
source_text = helper.load_data(source_path)
target_text = helper.load_data(target_path)

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ec2-user/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ec2-user/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


## Explore the Data
Play around with view_sentence_range to view different parts of the data.

In [2]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in source_text.split()})))

sentences = source_text.split('\n')
word_counts = [len(sentence.split()) for sentence in sentences]
print('Number of sentences: {}'.format(len(sentences)))
print('Average number of words in a sentence: {}'.format(np.average(word_counts)))

print()
print('English sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(source_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))
print()
print('French sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(target_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 227
Number of sentences: 137861
Average number of words in a sentence: 13.225277634719028

English sentences 0 to 10:
new jersey is sometimes quiet during autumn , and it is snowy in april .
the united states is usually chilly during july , and it is usually freezing in november .
california is usually quiet during march , and it is usually hot in june .
the united states is sometimes mild during june , and it is cold in september .
your least liked fruit is the grape , but my least liked is the apple .
his favorite fruit is the orange , but my favorite is the grape .
paris is relaxing during december , but it is usually chilly in july .
new jersey is busy during spring , and it is never hot in march .
our least liked fruit is the lemon , but my least liked is the grape .
the united states is sometimes busy during january , and it is sometimes warm in november .

French sentences 0 to 10:
new jersey est parfois calme pendant l' automne 

## Implement Preprocessing Function
### Text to Word Ids
As you did with other RNNs, you must turn the text into a number so the computer can understand it. In the function `text_to_ids()`, you'll turn `source_text` and `target_text` from words to ids.  However, you need to add the `<EOS>` word id at the end of `target_text`.  This will help the neural network predict when the sentence should end.

You can get the `<EOS>` word id by doing:
```python
target_vocab_to_int['<EOS>']
```
You can get other word ids using `source_vocab_to_int` and `target_vocab_to_int`.

In [3]:
test_source_text = 'new jersey is sometimes quiet during autumn , and it is snowy in april .\nthe united states is usually chilly during july , and it is usually freezing in november .\ncalifornia is usually quiet during march , and it is usually hot in june .\nthe united states is sometimes mild during june , and it is cold in september .'
test_target_text = 'new jersey est parfois calme pendant l\' automne , et il est neigeux en avril .\nles états-unis est généralement froid en juillet , et il gèle habituellement en novembre .\ncalifornia est généralement calme en mars , et il est généralement chaud en juin .\nles états-unis est parfois légère en juin , et il fait froid en septembre .'

test_source_text = test_source_text.lower()
test_target_text = test_target_text.lower()

source_vocab_to_int, source_int_to_vocab = helper.create_lookup_tables(test_source_text)
source_vocab_to_int.items()

dict_items([('<PAD>', 0), ('<EOS>', 1), ('<UNK>', 2), ('<GO>', 3), ('united', 4), ('freezing', 5), ('quiet', 6), (',', 7), ('april', 8), ('june', 9), ('new', 10), ('july', 11), ('jersey', 12), ('it', 13), ('.', 14), ('in', 15), ('november', 16), ('march', 17), ('the', 18), ('sometimes', 19), ('is', 20), ('september', 21), ('snowy', 22), ('and', 23), ('hot', 24), ('during', 25), ('usually', 26), ('mild', 27), ('cold', 28), ('california', 29), ('autumn', 30), ('states', 31), ('chilly', 32)])

In [4]:
def text_to_ids(source_text, target_text, source_vocab_to_int, target_vocab_to_int):
    """
    Convert source and target text to proper word ids
    :param source_text: String that contains all the source text.
    :param target_text: String that contains all the target text.
    :param source_vocab_to_int: Dictionary to go from the source words to an id
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: A tuple of lists (source_id_text, target_id_text)
    """
    # TODO: Implement Function
    
    #splitting to desired format and adding <EOS>
    source_list = [s.split() for s in source_text.split('\n')]
    target_list = [s.split() for s in target_text.split('\n')]
    
    #adding <EOS> TODO:inline way
    for s in target_list:
        s.append('<EOS>')
    
    #converting vocab to int
    source_id_text = [[source_vocab_to_int[v] for v in s] for s in source_list]
    target_id_text = [[target_vocab_to_int[v] for v in s] for s in target_list]
    
    return source_id_text, target_id_text

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_text_to_ids(text_to_ids)

Tests Passed


### Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [5]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
helper.preprocess_and_save_data(source_path, target_path, text_to_ids)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np
import helper
import problem_unittests as tests

(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()

### Check the Version of TensorFlow and Access to GPU
This will check to make sure you have the correct version of TensorFlow and access to a GPU

In [7]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf
from tensorflow.python.layers.core import Dense

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.1'), 'Please use TensorFlow version 1.1 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.5.0
Default GPU Device: /device:GPU:0


## Build the Neural Network
You'll build the components necessary to build a Sequence-to-Sequence model by implementing the following functions below:
- `model_inputs`
- `process_decoder_input`
- `encoding_layer`
- `decoding_layer_train`
- `decoding_layer_infer`
- `decoding_layer`
- `seq2seq_model`

### Input
Implement the `model_inputs()` function to create TF Placeholders for the Neural Network. It should create the following placeholders:

- Input text placeholder named "input" using the TF Placeholder name parameter with rank 2.
- Targets placeholder with rank 2.
- Learning rate placeholder with rank 0.
- Keep probability placeholder named "keep_prob" using the TF Placeholder name parameter with rank 0.
- Target sequence length placeholder named "target_sequence_length" with rank 1
- Max target sequence length tensor named "max_target_len" getting its value from applying tf.reduce_max on the target_sequence_length placeholder. Rank 0.
- Source sequence length placeholder named "source_sequence_length" with rank 1

Return the placeholders in the following the tuple (input, targets, learning rate, keep probability, target sequence length, max target sequence length, source sequence length)

In [8]:
def model_inputs():
    """
    Create TF Placeholders for input, targets, learning rate, and lengths of source and target sequences.
    :return: Tuple (input, targets, learning rate, keep probability, target sequence length,
    max target sequence length, source sequence length)
    """
    # TODO: Implement Function
    inputt = tf.placeholder(dtype=tf.int32,shape=[None,None],name="input")
    targets = tf.placeholder(dtype=tf.int32,shape=[None,None],name="targets")
    lrate = tf.placeholder(dtype=tf.float32,shape=None,name="lrate")
    keep_prob = tf.placeholder(dtype=tf.float32,shape=None,name="keep_prob")
    target_sequence_length = tf.placeholder(dtype=tf.int32,shape=[None],name="target_sequence_length")
    max_target_len =tf.reduce_max(target_sequence_length)
    source_sequence_length = tf.placeholder(dtype=tf.int32,shape=[None], name="source_sequence_length")
    return inputt, targets, lrate, keep_prob, target_sequence_length, max_target_len, source_sequence_length


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_model_inputs(model_inputs)

Tests Passed


### Process Decoder Input
Implement `process_decoder_input` by removing the last word id from each batch in `target_data` and concat the GO ID to the begining of each batch.

In [9]:
def process_decoder_input(target_data, target_vocab_to_int, batch_size):
    """
    Preprocess target data for encoding
    :param target_data: Target Placehoder
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param batch_size: Batch Size
    :return: Preprocessed target data
    """
    # TODO: Implement Function
    data_sliced = tf.strided_slice(target_data, [0,0], [batch_size, -1], [1,1]) #removing the last word
    data_filled = tf.concat([tf.fill([batch_size,1], target_vocab_to_int['<GO>']), data_sliced], 1) #adding "<GO>"
    
    return data_filled

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_process_encoding_input(process_decoder_input)

Tests Passed


### Encoding
Implement `encoding_layer()` to create a Encoder RNN layer:
 * Embed the encoder input using [`tf.contrib.layers.embed_sequence`](https://www.tensorflow.org/api_docs/python/tf/contrib/layers/embed_sequence)
 * Construct a [stacked](https://github.com/tensorflow/tensorflow/blob/6947f65a374ebf29e74bb71e36fd82760056d82c/tensorflow/docs_src/tutorials/recurrent.md#stacking-multiple-lstms) [`tf.contrib.rnn.LSTMCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/LSTMCell) wrapped in a [`tf.contrib.rnn.DropoutWrapper`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/DropoutWrapper)
 * Pass cell and embedded input to [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)

In [10]:
from imp import reload
reload(tests)

def encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob, 
                   source_sequence_length, source_vocab_size, 
                   encoding_embedding_size):
    """
    Create encoding layer
    :param rnn_inputs: Inputs for the RNN
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param keep_prob: Dropout keep probability
    :param source_sequence_length: a list of the lengths of each sequence in the batch
    :param source_vocab_size: vocabulary size of source data
    :param encoding_embedding_size: embedding size of source data
    :return: tuple (RNN output, RNN state)
    """
    # TODO: Implement Function
    
    # Encoder embedding
    enc_embed_input = tf.contrib.layers.embed_sequence(rnn_inputs, source_vocab_size, encoding_embedding_size)

    # RNN cell
    def rnn_cell(rnn_size):
        rnn_cell = tf.contrib.rnn.LSTMCell(rnn_size) # an initializer could help to reproductability
        rnn_cell = tf.contrib.rnn.DropoutWrapper(rnn_cell,output_keep_prob=keep_prob)
        return rnn_cell

    enc_cell = tf.contrib.rnn.MultiRNNCell([rnn_cell(rnn_size) for _ in range(num_layers)])
    rnn_output, rnn_state = tf.nn.dynamic_rnn(enc_cell, 
                                              enc_embed_input, 
                                              sequence_length=source_sequence_length, 
                                              dtype=tf.float32)
    
    return rnn_output, rnn_state

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_encoding_layer(encoding_layer)

Tests Passed


### Decoding - Training
Create a training decoding layer:
* Create a [`tf.contrib.seq2seq.TrainingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/TrainingHelper) 
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [11]:

def decoding_layer_train(encoder_state, dec_cell, dec_embed_input, 
                         target_sequence_length, max_summary_length, 
                         output_layer, keep_prob):
    """
    Create a decoding layer for training
    :param encoder_state: Encoder State
    :param dec_cell: Decoder RNN Cell
    :param dec_embed_input: Decoder embedded input
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_summary_length: The length of the longest sequence in the batch
    :param output_layer: Function to apply the output layer
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing training logits and sample_id
    """
    # TODO: Implement Function
    
    # Helper is used by BasicDecoder to read inputs.
    helper = tf.contrib.seq2seq.TrainingHelper(inputs=dec_embed_input, sequence_length=target_sequence_length)

    # BasicDecoder
    basic_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell,helper, encoder_state,output_layer) 

    # Dynamic Decoder
    dynamic_decoder = tf.contrib.seq2seq.dynamic_decode(basic_decoder,impute_finished=True,
                                                                   maximum_iterations=max_summary_length)[0]
    return dynamic_decoder



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_train(decoding_layer_train)

Tests Passed


### Decoding - Inference
Create inference decoder:
* Create a [`tf.contrib.seq2seq.GreedyEmbeddingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/GreedyEmbeddingHelper)
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [12]:
def decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id,
                         end_of_sequence_id, max_target_sequence_length,
                         vocab_size, output_layer, batch_size, keep_prob):
    """
    Create a decoding layer for inference
    :param encoder_state: Encoder state
    :param dec_cell: Decoder RNN Cell
    :param dec_embeddings: Decoder embeddings
    :param start_of_sequence_id: GO ID
    :param end_of_sequence_id: EOS Id
    :param max_target_sequence_length: Maximum length of target sequences
    :param vocab_size: Size of decoder/target vocabulary
    :param decoding_scope: TenorFlow Variable Scope for decoding
    :param output_layer: Function to apply the output layer
    :param batch_size: Batch size
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing inference logits and sample_id
    """
    # TODO: Implement Function
    start_tokens = tf.tile([start_of_sequence_id], [batch_size])
    embedding_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(
        dec_embeddings, 
        start_tokens, 
        end_of_sequence_id
    )
    basic_decoder = tf.contrib.seq2seq.BasicDecoder(
        dec_cell, 
        embedding_helper, 
        encoder_state, 
        output_layer
    )
    inference_decoder_output = tf.contrib.seq2seq.dynamic_decode(
        basic_decoder,
        impute_finished=True,
        maximum_iterations=max_target_sequence_length
    )[0]
    
    return inference_decoder_output

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_infer(decoding_layer_infer)

Tests Passed


### Build the Decoding Layer
Implement `decoding_layer()` to create a Decoder RNN layer.

* Embed the target sequences
* Construct the decoder LSTM cell (just like you constructed the encoder cell above)
* Create an output layer to map the outputs of the decoder to the elements of our vocabulary
* Use the your `decoding_layer_train(encoder_state, dec_cell, dec_embed_input, target_sequence_length, max_target_sequence_length, output_layer, keep_prob)` function to get the training logits.
* Use your `decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id, end_of_sequence_id, max_target_sequence_length, vocab_size, output_layer, batch_size, keep_prob)` function to get the inference logits.

Note: You'll need to use [tf.variable_scope](https://www.tensorflow.org/api_docs/python/tf/variable_scope) to share variables between training and inference.

In [13]:
def decoding_layer(dec_input, encoder_state,
                   target_sequence_length, max_target_sequence_length,
                   rnn_size,
                   num_layers, target_vocab_to_int, target_vocab_size,
                   batch_size, keep_prob, decoding_embedding_size):
    """
    Create decoding layer
    :param dec_input: Decoder input
    :param encoder_state: Encoder state
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_target_sequence_length: Maximum length of target sequences
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param target_vocab_size: Size of target vocabulary
    :param batch_size: The size of the batch
    :param keep_prob: Dropout keep probability
    :param decoding_embedding_size: Decoding embedding size
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    # TODO: Implement Function
    def rnn_cell(rnn_size, keep_prob):
        lstm = tf.contrib.rnn.LSTMCell(rnn_size)
        lstm_drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)

        return lstm_drop

    stacked_lstm = tf.contrib.rnn.MultiRNNCell([rnn_cell(rnn_size, keep_prob) for i in range(num_layers)])
    
    dec_embeddings = tf.Variable(tf.random_uniform([target_vocab_size, decoding_embedding_size]))
    dec_embed_input = tf.nn.embedding_lookup(dec_embeddings, dec_input)

    dense_layer = Dense(
                    target_vocab_size,
                    kernel_initializer = tf.truncated_normal_initializer(mean = 0.0, stddev=0.1)
                  )
    
    with tf.variable_scope("decode") as scope:
        trainig_decoder_output = decoding_layer_train(
            encoder_state, 
            stacked_lstm, 
            dec_embed_input, 
            target_sequence_length, 
            max_target_sequence_length, 
            dense_layer, 
            keep_prob
        )
        
    #reusing the parameters trained    
    with tf.variable_scope("decode", reuse=True):
        inference_decoder_output = decoding_layer_infer(
            encoder_state, 
            stacked_lstm, 
            dec_embeddings, 
            target_vocab_to_int['<GO>'], 
            target_vocab_to_int['<EOS>'], 
            max_target_sequence_length, 
            target_vocab_size, 
            dense_layer, 
            batch_size, 
            keep_prob
        )
    
    return trainig_decoder_output, inference_decoder_output



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer(decoding_layer)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:

- Encode the input using your `encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob,  source_sequence_length, source_vocab_size, encoding_embedding_size)`.
- Process target data using your `process_decoder_input(target_data, target_vocab_to_int, batch_size)` function.
- Decode the encoded input using your `decoding_layer(dec_input, enc_state, target_sequence_length, max_target_sentence_length, rnn_size, num_layers, target_vocab_to_int, target_vocab_size, batch_size, keep_prob, dec_embedding_size)` function.

In [14]:
def seq2seq_model(input_data, target_data, keep_prob, batch_size,
                  source_sequence_length, target_sequence_length,
                  max_target_sentence_length,
                  source_vocab_size, target_vocab_size,
                  enc_embedding_size, dec_embedding_size,
                  rnn_size, num_layers, target_vocab_to_int):
    """
    Build the Sequence-to-Sequence part of the neural network
    :param input_data: Input placeholder
    :param target_data: Target placeholder
    :param keep_prob: Dropout keep probability placeholder
    :param batch_size: Batch Size
    :param source_sequence_length: Sequence Lengths of source sequences in the batch
    :param target_sequence_length: Sequence Lengths of target sequences in the batch
    :param source_vocab_size: Source vocabulary size
    :param target_vocab_size: Target vocabulary size
    :param enc_embedding_size: Decoder embedding size
    :param dec_embedding_size: Encoder embedding size
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    # TODO: Implement Function
    outputs, state = encoding_layer(
        input_data,
        rnn_size,
        num_layers,
        keep_prob,
        source_sequence_length,
        source_vocab_size,
        enc_embedding_size
    )
    
    processed_data = process_decoder_input(
        target_data, 
        target_vocab_to_int, 
        batch_size
    )
    
    training_decoder_output, inference_decoder_output = decoding_layer(
        processed_data,
        state,
        target_sequence_length,
        max_target_sentence_length,
        rnn_size, 
        num_layers,
        target_vocab_to_int,
        target_vocab_size,
        batch_size,
        keep_prob,
        dec_embedding_size
    )
    
    return training_decoder_output, inference_decoder_output


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_seq2seq_model(seq2seq_model)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `num_layers` to the number of layers.
- Set `encoding_embedding_size` to the size of the embedding for the encoder.
- Set `decoding_embedding_size` to the size of the embedding for the decoder.
- Set `learning_rate` to the learning rate.
- Set `keep_probability` to the Dropout keep probability
- Set `display_step` to state how many steps between each debug output statement

In [15]:
# Number of Epochs
epochs = 10
# Batch Size
batch_size = 512
# RNN Size
rnn_size = 256
# Number of Layers
num_layers = 2
# Embedding Size
encoding_embedding_size = 128
decoding_embedding_size = 128
# Learning Rate
learning_rate = 0.001
# Dropout Keep Probability
keep_probability = 0.7
display_step = True

### Build the Graph
Build the graph using the neural network you implemented.

In [16]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
save_path = 'checkpoints/dev'
(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()
max_target_sentence_length = max([len(sentence) for sentence in source_int_text])

train_graph = tf.Graph()
with train_graph.as_default():
    input_data, targets, lr, keep_prob, target_sequence_length, max_target_sequence_length, source_sequence_length = model_inputs()

    #sequence_length = tf.placeholder_with_default(max_target_sentence_length, None, name='sequence_length')
    input_shape = tf.shape(input_data)

    train_logits, inference_logits = seq2seq_model(tf.reverse(input_data, [-1]),
                                                   targets,
                                                   keep_prob,
                                                   batch_size,
                                                   source_sequence_length,
                                                   target_sequence_length,
                                                   max_target_sequence_length,
                                                   len(source_vocab_to_int),
                                                   len(target_vocab_to_int),
                                                   encoding_embedding_size,
                                                   decoding_embedding_size,
                                                   rnn_size,
                                                   num_layers,
                                                   target_vocab_to_int)


    training_logits = tf.identity(train_logits.rnn_output, name='logits')
    inference_logits = tf.identity(inference_logits.sample_id, name='predictions')

    masks = tf.sequence_mask(target_sequence_length, max_target_sequence_length, dtype=tf.float32, name='masks')

    with tf.name_scope("optimization"):
        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(
            training_logits,
            targets,
            masks)

        # Optimizer
        optimizer = tf.train.AdamOptimizer(lr)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)


Batch and pad the source and target sequences

In [17]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def pad_sentence_batch(sentence_batch, pad_int):
    """Pad sentences with <PAD> so that each sentence of a batch has the same length"""
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [pad_int] * (max_sentence - len(sentence)) for sentence in sentence_batch]


def get_batches(sources, targets, batch_size, source_pad_int, target_pad_int):
    """Batch targets, sources, and the lengths of their sentences together"""
    for batch_i in range(0, len(sources)//batch_size):
        start_i = batch_i * batch_size

        # Slice the right amount for the batch
        sources_batch = sources[start_i:start_i + batch_size]
        targets_batch = targets[start_i:start_i + batch_size]

        # Pad
        pad_sources_batch = np.array(pad_sentence_batch(sources_batch, source_pad_int))
        pad_targets_batch = np.array(pad_sentence_batch(targets_batch, target_pad_int))

        # Need the lengths for the _lengths parameters
        pad_targets_lengths = []
        for target in pad_targets_batch:
            pad_targets_lengths.append(len(target))

        pad_source_lengths = []
        for source in pad_sources_batch:
            pad_source_lengths.append(len(source))

        yield pad_sources_batch, pad_targets_batch, pad_source_lengths, pad_targets_lengths


### Train
Train the neural network on the preprocessed data. If you have a hard time getting a good loss, check the forms to see if anyone is having the same problem.

In [18]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def get_accuracy(target, logits):
    """
    Calculate accuracy
    """
    max_seq = max(target.shape[1], logits.shape[1])
    if max_seq - target.shape[1]:
        target = np.pad(
            target,
            [(0,0),(0,max_seq - target.shape[1])],
            'constant')
    if max_seq - logits.shape[1]:
        logits = np.pad(
            logits,
            [(0,0),(0,max_seq - logits.shape[1])],
            'constant')

    return np.mean(np.equal(target, logits))

# Split data to training and validation sets
train_source = source_int_text[batch_size:]
train_target = target_int_text[batch_size:]
valid_source = source_int_text[:batch_size]
valid_target = target_int_text[:batch_size]
(valid_sources_batch, valid_targets_batch, valid_sources_lengths, valid_targets_lengths ) = next(get_batches(valid_source,
                                                                                                             valid_target,
                                                                                                             batch_size,
                                                                                                             source_vocab_to_int['<PAD>'],
                                                                                                             target_vocab_to_int['<PAD>']))                                                                                                  
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(epochs):
        for batch_i, (source_batch, target_batch, sources_lengths, targets_lengths) in enumerate(
                get_batches(train_source, train_target, batch_size,
                            source_vocab_to_int['<PAD>'],
                            target_vocab_to_int['<PAD>'])):

            _, loss = sess.run(
                [train_op, cost],
                {input_data: source_batch,
                 targets: target_batch,
                 lr: learning_rate,
                 target_sequence_length: targets_lengths,
                 source_sequence_length: sources_lengths,
                 keep_prob: keep_probability})


            if batch_i % display_step == 0 and batch_i > 0:


                batch_train_logits = sess.run(
                    inference_logits,
                    {input_data: source_batch,
                     source_sequence_length: sources_lengths,
                     target_sequence_length: targets_lengths,
                     keep_prob: 1.0})


                batch_valid_logits = sess.run(
                    inference_logits,
                    {input_data: valid_sources_batch,
                     source_sequence_length: valid_sources_lengths,
                     target_sequence_length: valid_targets_lengths,
                     keep_prob: 1.0})

                train_acc = get_accuracy(target_batch, batch_train_logits)

                valid_acc = get_accuracy(valid_targets_batch, batch_valid_logits)

                print('Epoch {:>3} Batch {:>4}/{} - Train Accuracy: {:>6.4f}, Validation Accuracy: {:>6.4f}, Loss: {:>6.4f}'
                      .format(epoch_i, batch_i, len(source_int_text) // batch_size, train_acc, valid_acc, loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_path)
    print('Model Trained and Saved')

Epoch   0 Batch    1/269 - Train Accuracy: 0.2329, Validation Accuracy: 0.3096, Loss: 5.5305
Epoch   0 Batch    2/269 - Train Accuracy: 0.2655, Validation Accuracy: 0.3096, Loss: 4.9668
Epoch   0 Batch    3/269 - Train Accuracy: 0.2444, Validation Accuracy: 0.3097, Loss: 4.6621
Epoch   0 Batch    4/269 - Train Accuracy: 0.2418, Validation Accuracy: 0.3185, Loss: 4.7250
Epoch   0 Batch    5/269 - Train Accuracy: 0.2575, Validation Accuracy: 0.3322, Loss: 4.5343
Epoch   0 Batch    6/269 - Train Accuracy: 0.3118, Validation Accuracy: 0.3411, Loss: 4.0644
Epoch   0 Batch    7/269 - Train Accuracy: 0.3108, Validation Accuracy: 0.3427, Loss: 3.9879
Epoch   0 Batch    8/269 - Train Accuracy: 0.2784, Validation Accuracy: 0.3428, Loss: 4.0208
Epoch   0 Batch    9/269 - Train Accuracy: 0.3057, Validation Accuracy: 0.3463, Loss: 3.8164
Epoch   0 Batch   10/269 - Train Accuracy: 0.2820, Validation Accuracy: 0.3530, Loss: 3.7821
Epoch   0 Batch   11/269 - Train Accuracy: 0.3166, Validation Accuracy

Epoch   0 Batch   90/269 - Train Accuracy: 0.4286, Validation Accuracy: 0.4909, Loss: 2.0921
Epoch   0 Batch   91/269 - Train Accuracy: 0.4317, Validation Accuracy: 0.4662, Loss: 1.9535
Epoch   0 Batch   92/269 - Train Accuracy: 0.4674, Validation Accuracy: 0.4908, Loss: 1.9552
Epoch   0 Batch   93/269 - Train Accuracy: 0.4863, Validation Accuracy: 0.4916, Loss: 1.8655
Epoch   0 Batch   94/269 - Train Accuracy: 0.4544, Validation Accuracy: 0.4852, Loss: 1.9600
Epoch   0 Batch   95/269 - Train Accuracy: 0.4575, Validation Accuracy: 0.4824, Loss: 1.9056
Epoch   0 Batch   96/269 - Train Accuracy: 0.4805, Validation Accuracy: 0.4972, Loss: 1.8897
Epoch   0 Batch   97/269 - Train Accuracy: 0.4630, Validation Accuracy: 0.4924, Loss: 1.9024
Epoch   0 Batch   98/269 - Train Accuracy: 0.4665, Validation Accuracy: 0.4798, Loss: 1.8347
Epoch   0 Batch   99/269 - Train Accuracy: 0.4491, Validation Accuracy: 0.5016, Loss: 1.9855
Epoch   0 Batch  100/269 - Train Accuracy: 0.4917, Validation Accuracy

Epoch   0 Batch  179/269 - Train Accuracy: 0.5060, Validation Accuracy: 0.5202, Loss: 1.2761
Epoch   0 Batch  180/269 - Train Accuracy: 0.5092, Validation Accuracy: 0.5138, Loss: 1.2548
Epoch   0 Batch  181/269 - Train Accuracy: 0.5061, Validation Accuracy: 0.5319, Loss: 1.2667
Epoch   0 Batch  182/269 - Train Accuracy: 0.4928, Validation Accuracy: 0.5180, Loss: 1.2742
Epoch   0 Batch  183/269 - Train Accuracy: 0.5734, Validation Accuracy: 0.5390, Loss: 1.0947
Epoch   0 Batch  184/269 - Train Accuracy: 0.4992, Validation Accuracy: 0.5444, Loss: 1.2959
Epoch   0 Batch  185/269 - Train Accuracy: 0.4970, Validation Accuracy: 0.5178, Loss: 1.2378
Epoch   0 Batch  186/269 - Train Accuracy: 0.4857, Validation Accuracy: 0.5455, Loss: 1.2965
Epoch   0 Batch  187/269 - Train Accuracy: 0.5327, Validation Accuracy: 0.5496, Loss: 1.2143
Epoch   0 Batch  188/269 - Train Accuracy: 0.5190, Validation Accuracy: 0.5379, Loss: 1.2024
Epoch   0 Batch  189/269 - Train Accuracy: 0.5283, Validation Accuracy

Epoch   1 Batch    1/269 - Train Accuracy: 0.5058, Validation Accuracy: 0.5443, Loss: 0.9609
Epoch   1 Batch    2/269 - Train Accuracy: 0.5118, Validation Accuracy: 0.5499, Loss: 0.9451
Epoch   1 Batch    3/269 - Train Accuracy: 0.5295, Validation Accuracy: 0.5588, Loss: 0.9514
Epoch   1 Batch    4/269 - Train Accuracy: 0.5308, Validation Accuracy: 0.5563, Loss: 0.9635
Epoch   1 Batch    5/269 - Train Accuracy: 0.5126, Validation Accuracy: 0.5574, Loss: 0.9606
Epoch   1 Batch    6/269 - Train Accuracy: 0.5609, Validation Accuracy: 0.5629, Loss: 0.8786
Epoch   1 Batch    7/269 - Train Accuracy: 0.5431, Validation Accuracy: 0.5589, Loss: 0.9052
Epoch   1 Batch    8/269 - Train Accuracy: 0.5194, Validation Accuracy: 0.5551, Loss: 0.9625
Epoch   1 Batch    9/269 - Train Accuracy: 0.5333, Validation Accuracy: 0.5605, Loss: 0.9316
Epoch   1 Batch   10/269 - Train Accuracy: 0.5332, Validation Accuracy: 0.5608, Loss: 0.9303
Epoch   1 Batch   11/269 - Train Accuracy: 0.5339, Validation Accuracy

Epoch   1 Batch   90/269 - Train Accuracy: 0.5362, Validation Accuracy: 0.5862, Loss: 0.7963
Epoch   1 Batch   91/269 - Train Accuracy: 0.5633, Validation Accuracy: 0.5874, Loss: 0.7317
Epoch   1 Batch   92/269 - Train Accuracy: 0.5728, Validation Accuracy: 0.5848, Loss: 0.7386
Epoch   1 Batch   93/269 - Train Accuracy: 0.5850, Validation Accuracy: 0.5869, Loss: 0.7158
Epoch   1 Batch   94/269 - Train Accuracy: 0.5675, Validation Accuracy: 0.5867, Loss: 0.7645
Epoch   1 Batch   95/269 - Train Accuracy: 0.5698, Validation Accuracy: 0.5893, Loss: 0.7477
Epoch   1 Batch   96/269 - Train Accuracy: 0.5806, Validation Accuracy: 0.5919, Loss: 0.7467
Epoch   1 Batch   97/269 - Train Accuracy: 0.5561, Validation Accuracy: 0.5882, Loss: 0.7365
Epoch   1 Batch   98/269 - Train Accuracy: 0.5857, Validation Accuracy: 0.5873, Loss: 0.7458
Epoch   1 Batch   99/269 - Train Accuracy: 0.5616, Validation Accuracy: 0.5920, Loss: 0.7705
Epoch   1 Batch  100/269 - Train Accuracy: 0.5759, Validation Accuracy

...

Epoch   8 Batch  179/269 - Train Accuracy: 0.9280, Validation Accuracy: 0.9416, Loss: 0.0470
Epoch   8 Batch  180/269 - Train Accuracy: 0.9530, Validation Accuracy: 0.9418, Loss: 0.0408
Epoch   8 Batch  181/269 - Train Accuracy: 0.9446, Validation Accuracy: 0.9420, Loss: 0.0523
Epoch   8 Batch  182/269 - Train Accuracy: 0.9493, Validation Accuracy: 0.9376, Loss: 0.0461
Epoch   8 Batch  183/269 - Train Accuracy: 0.9477, Validation Accuracy: 0.9376, Loss: 0.0389
Epoch   8 Batch  184/269 - Train Accuracy: 0.9438, Validation Accuracy: 0.9379, Loss: 0.0464
Epoch   8 Batch  185/269 - Train Accuracy: 0.9520, Validation Accuracy: 0.9410, Loss: 0.0454
Epoch   8 Batch  186/269 - Train Accuracy: 0.9393, Validation Accuracy: 0.9438, Loss: 0.0439
Epoch   8 Batch  187/269 - Train Accuracy: 0.9382, Validation Accuracy: 0.9461, Loss: 0.0466
Epoch   8 Batch  188/269 - Train Accuracy: 0.9416, Validation Accuracy: 0.9419, Loss: 0.0421
Epoch   8 Batch  189/269 - Train Accuracy: 0.9407, Validation Accuracy

Epoch   9 Batch    1/269 - Train Accuracy: 0.9407, Validation Accuracy: 0.9482, Loss: 0.0416
Epoch   9 Batch    2/269 - Train Accuracy: 0.9430, Validation Accuracy: 0.9443, Loss: 0.0482
Epoch   9 Batch    3/269 - Train Accuracy: 0.9497, Validation Accuracy: 0.9362, Loss: 0.0443
Epoch   9 Batch    4/269 - Train Accuracy: 0.9319, Validation Accuracy: 0.9392, Loss: 0.0475
Epoch   9 Batch    5/269 - Train Accuracy: 0.9499, Validation Accuracy: 0.9316, Loss: 0.0438
Epoch   9 Batch    6/269 - Train Accuracy: 0.9484, Validation Accuracy: 0.9372, Loss: 0.0438
Epoch   9 Batch    7/269 - Train Accuracy: 0.9407, Validation Accuracy: 0.9426, Loss: 0.0422
Epoch   9 Batch    8/269 - Train Accuracy: 0.9423, Validation Accuracy: 0.9419, Loss: 0.0499
Epoch   9 Batch    9/269 - Train Accuracy: 0.9447, Validation Accuracy: 0.9333, Loss: 0.0452
Epoch   9 Batch   10/269 - Train Accuracy: 0.9419, Validation Accuracy: 0.9408, Loss: 0.0432
Epoch   9 Batch   11/269 - Train Accuracy: 0.9532, Validation Accuracy

Epoch   9 Batch   90/269 - Train Accuracy: 0.9445, Validation Accuracy: 0.9486, Loss: 0.0413
Epoch   9 Batch   91/269 - Train Accuracy: 0.9596, Validation Accuracy: 0.9498, Loss: 0.0388
Epoch   9 Batch   92/269 - Train Accuracy: 0.9630, Validation Accuracy: 0.9442, Loss: 0.0355
Epoch   9 Batch   93/269 - Train Accuracy: 0.9443, Validation Accuracy: 0.9396, Loss: 0.0389
Epoch   9 Batch   94/269 - Train Accuracy: 0.9466, Validation Accuracy: 0.9393, Loss: 0.0455
Epoch   9 Batch   95/269 - Train Accuracy: 0.9488, Validation Accuracy: 0.9487, Loss: 0.0396
Epoch   9 Batch   96/269 - Train Accuracy: 0.9303, Validation Accuracy: 0.9506, Loss: 0.0459
Epoch   9 Batch   97/269 - Train Accuracy: 0.9516, Validation Accuracy: 0.9441, Loss: 0.0417
Epoch   9 Batch   98/269 - Train Accuracy: 0.9563, Validation Accuracy: 0.9477, Loss: 0.0383
Epoch   9 Batch   99/269 - Train Accuracy: 0.9322, Validation Accuracy: 0.9390, Loss: 0.0389
Epoch   9 Batch  100/269 - Train Accuracy: 0.9498, Validation Accuracy

Epoch   9 Batch  179/269 - Train Accuracy: 0.9426, Validation Accuracy: 0.9466, Loss: 0.0380
Epoch   9 Batch  180/269 - Train Accuracy: 0.9581, Validation Accuracy: 0.9436, Loss: 0.0392
Epoch   9 Batch  181/269 - Train Accuracy: 0.9410, Validation Accuracy: 0.9504, Loss: 0.0432
Epoch   9 Batch  182/269 - Train Accuracy: 0.9482, Validation Accuracy: 0.9484, Loss: 0.0356
Epoch   9 Batch  183/269 - Train Accuracy: 0.9490, Validation Accuracy: 0.9505, Loss: 0.0338
Epoch   9 Batch  184/269 - Train Accuracy: 0.9555, Validation Accuracy: 0.9519, Loss: 0.0397
Epoch   9 Batch  185/269 - Train Accuracy: 0.9508, Validation Accuracy: 0.9498, Loss: 0.0378
Epoch   9 Batch  186/269 - Train Accuracy: 0.9485, Validation Accuracy: 0.9488, Loss: 0.0365
Epoch   9 Batch  187/269 - Train Accuracy: 0.9477, Validation Accuracy: 0.9446, Loss: 0.0392
Epoch   9 Batch  188/269 - Train Accuracy: 0.9501, Validation Accuracy: 0.9398, Loss: 0.0359
Epoch   9 Batch  189/269 - Train Accuracy: 0.9500, Validation Accuracy

Model Trained and Saved


### Save Parameters
Save the `batch_size` and `save_path` parameters for inference.

In [19]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params(save_path)

# Checkpoint

In [20]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, (source_vocab_to_int, target_vocab_to_int), (source_int_to_vocab, target_int_to_vocab) = helper.load_preprocess()
load_path = helper.load_params()

## Sentence to Sequence
To feed a sentence into the model for translation, you first need to preprocess it.  Implement the function `sentence_to_seq()` to preprocess new sentences.

- Convert the sentence to lowercase
- Convert words into ids using `vocab_to_int`
 - Convert words not in the vocabulary, to the `<UNK>` word id.

In [21]:
def sentence_to_seq(sentence, vocab_to_int):
    """
    Convert a sentence to a sequence of ids
    :param sentence: String
    :param vocab_to_int: Dictionary to go from the words to an id
    :return: List of word ids
    """
    # TODO: Implement Function
    return [vocab_to_int.get(word, vocab_to_int['<UNK>']) for word in sentence.lower().split()]


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_sentence_to_seq(sentence_to_seq)

Tests Passed


## Translate
This will translate `translate_sentence` from English to French.

In [22]:
translate_sentence = 'he saw a old yellow truck .'


"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
translate_sentence = sentence_to_seq(translate_sentence, source_vocab_to_int)

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_path + '.meta')
    loader.restore(sess, load_path)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    target_sequence_length = loaded_graph.get_tensor_by_name('target_sequence_length:0')
    source_sequence_length = loaded_graph.get_tensor_by_name('source_sequence_length:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')

    translate_logits = sess.run(logits, {input_data: [translate_sentence]*batch_size,
                                         target_sequence_length: [len(translate_sentence)*2]*batch_size,
                                         source_sequence_length: [len(translate_sentence)]*batch_size,
                                         keep_prob: 1.0})[0]

print('Input')
print('  Word Ids:      {}'.format([i for i in translate_sentence]))
print('  English Words: {}'.format([source_int_to_vocab[i] for i in translate_sentence]))

print('\nPrediction')
print('  Word Ids:      {}'.format([i for i in translate_logits]))
print('  French Words: {}'.format(" ".join([target_int_to_vocab[i] for i in translate_logits])))


INFO:tensorflow:Restoring parameters from checkpoints/dev
Input
  Word Ids:      [190, 143, 140, 169, 112, 152, 96]
  English Words: ['he', 'saw', 'a', 'old', 'yellow', 'truck', '.']

Prediction
  Word Ids:      [307, 170, 288, 149, 331, 216, 1]
  French Words: il est vieux camion bleu . <EOS>


## Imperfect Translation
You might notice that some sentences translate better than others.  Since the dataset you're using only has a vocabulary of 227 English words of the thousands that you use, you're only going to see good results using these words.  For this project, you don't need a perfect translation. However, if you want to create a better translation model, you'll need better data.

You can train on the [WMT10 French-English corpus](http://www.statmt.org/wmt10/training-giga-fren.tar).  This dataset has more vocabulary and richer in topics discussed.  However, this will take you days to train, so make sure you've a GPU and the neural network is performing well on dataset we provided.  Just make sure you play with the WMT10 corpus after you've submitted this project.
## Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_language_translation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.